# COVID-19 cases

### Load python tools

In [1]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/Users/mhustiles/data/data/chromedriver')

### CDC testing capacity

In [2]:
tables = pd.read_html('https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/testing-in-us.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov\
%2Fcoronavirus%2F2019-ncov%2Ftesting-in-us.html',  header=0)

In [9]:
tables = pd.DataFrame(tables[2])

KeyError: 2

In [4]:
tables.columns = tables.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [ ]:
tables.rename(columns={"us_public_health_labs": "US Public Health Labs",\
                   "cdc_labs": "CDC Labs", 'date_collected':'Date'}, inplace=True)

In [7]:
tables

,total_tests_reported
0,13627379


In [6]:
tables['Date'] = pd.to_datetime(tables['Date'] + '/2020')

KeyError: 'Date'

In [ ]:
tables.head()

In [ ]:
tables['US Public Health Labs'] = tables['US Public Health Labs'].str.replace('‡','').str.replace('§','').fillna('0')

In [ ]:
tables['CDC Labs'] = tables['CDC Labs'].str.replace('§','').str.replace('‡','').fillna('0')

In [ ]:
# tables = tables[tables['Date'] <= '2020/03/10']

In [ ]:
tables_melt = pd.melt(tables, id_vars=['Date'], value_vars=['US Public Health Labs', 'CDC Labs'],
        var_name='lab_type', value_name='test_count')

In [ ]:
tables.to_csv('output/cdc_testing_table.csv')
tables_melt.to_csv('output/cdc_testing_table_melt.csv')
tables_melt.tail(10)

In [ ]:
tables_melt.head()

In [ ]:
tables_melt.head()

In [ ]:
chart = alt.Chart(tables_melt).mark_bar(size=10).encode(
    x=alt.X('Date:T', title='Date', axis=alt.Axis(tickCount=10)),
    y=alt.Y('test_count:Q', title=''),
    color=alt.Color('lab_type', legend=alt.Legend(orient="top"), title=''),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'lab_type',
      sort='ascending'
    )
).properties(width=800, title={
      "text": ["CDC: Daily U.S. coronavirus specimens tested, by lab type"], 
      "subtitle": ["Figures from last four days reported as 'pending'"],
      "color": "black",
      "subtitleColor": "gray"})

chart

In [ ]:
chart.save('visualization.png')

In [ ]:
chart.save('visualization.svg')

---

In [ ]:
url_cases = 'https://www.cdc.gov/coronavirus/2019-ncov/map-data-cases.csv'

In [ ]:
cases = pd.read_csv(url_cases, encoding='latin-1')

In [ ]:
cases.columns = cases.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [ ]:
cases['cases_reported'] = cases['cases_reported'].str.replace('None', '0')

In [ ]:
cases.drop(['unnamed:_5', 'unnamed:_6', 'unnamed:_7'], axis=1, inplace=True)

In [ ]:
cases.head()

---

### Johns Hopkins data

In [ ]:
# Cases: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1

# Deaths: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/0

# Cases countries: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/2

# Cases/time: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/cases_time_v3/FeatureServer/0

---

In [ ]:
# !tippecanoe --generate-ids --force -r1 -pk -pf -o \
# output/icus_geo.mbtiles \
# output/icus_geo.geojson